In [55]:
import re
import nltk
import string
import os
import gensim
import pymorphy2
morph = pymorphy2.MorphAnalyzer()
from gensim.models import Word2Vec
#from nltk.stem.snowball import RussianStemmer
#stemmer = RussianStemmer()

In [ ]:
def canonize_words(words: list) -> list:
    stop_words = ('быть', 'мой', 'наш', 'ваш', 'их', 'его', 'её', 'их',
                  'этот', 'тот', 'где', 'который', 'либо', 'нибудь', 'нет', 'да')
    grammars = {'NOUN': '_S',
                'VERB': '_V', 'INFN': '_V', 'GRND': '_V', 'PRTF': '_V', 'PRTS': '_V',
                'ADJF': '_A', 'ADJS': '_A',
                'ADVB': '_ADV',
                'PRED': '_PRAEDIC'}

    morph = pymorphy2.MorphAnalyzer()
    normalized = []
    for i in words:
        forms = morph.parse(i)
        try:
            form = max(forms, key=lambda x: (x.score, x.methods_stack[0][2]))
        except Exception:
            form = forms[0]
            print(form)
        if not (form.tag.POS in ['PREP', 'CONJ', 'PRCL', 'NPRO', 'NUMR']
                or 'Name' in form.tag
                or 'UNKN' in form.tag
                or form.normal_form in stop_words):  # 'ADJF'
            normalized.append(form.normal_form + grammars.get(form.tag.POS, ''))
    return normalized

Экземпляры класса MorphAnalyzer обычно занимают порядка 15Мб оперативной памяти (т.к. загружают в память словари, данные для предсказателя и т.д.); старайтесь организовать свой код так, чтобы создавать экземпляр MorphAnalyzer заранее и работать с этим единственным экземпляром в дальнейшем.

In [2]:
morph.parse('лох')

[Parse(word='лох', tag=OpencorporaTag('NOUN,anim,masc sing,nomn'), normal_form='лох', score=0.3333333333333333, methods_stack=((<DictionaryAnalyzer>, 'лох', 2, 0),)),
 Parse(word='лох', tag=OpencorporaTag('NOUN,inan,masc sing,nomn'), normal_form='лох', score=0.3333333333333333, methods_stack=((<DictionaryAnalyzer>, 'лох', 19, 0),)),
 Parse(word='лох', tag=OpencorporaTag('NOUN,inan,masc sing,accs'), normal_form='лох', score=0.3333333333333333, methods_stack=((<DictionaryAnalyzer>, 'лох', 19, 3),))]

In [35]:
p = morph.parse('лох')[2]
p.tag
p.tag.POS #chast rechi
p.tag.case #padesh
p.tag.number
p.tag.gender

'masc'

http://pymorphy2.readthedocs.io/en/latest/user/grammemes.html#grammeme-docs

In [11]:
p.normal_form

'лох'

In [16]:
p.tag.cyr_repr

'СУЩ,неод,мр ед,вн'

In [15]:
{'NOUN', 'inan'} in p.tag

True

In [20]:
p.inflect({'gent', 'plur'}).word

'лохов'

In [25]:
p.lexeme[1].word #vse slovoformy

'лоха'

In [23]:
p.make_agree_with_number(5).word

'лохов'

In [39]:
morph.parse('йопт')[0].tag.cyr_repr

'СУЩ,неод,мр ед,им'

##### Загружаем обученную на русской Википедии модель word2vec

In [42]:
wiki = gensim.models.KeyedVectors.load_word2vec_format('C:\\Users\\user\\Downloads\\vectors-ru_wiki\\vectors.bin', binary=True)
#ёmodel.similar_by_word('луна')
#model.most_similar(positive = ['луна'])

In [44]:
wiki.most_similar(positive = ['луна'])

[('планета', 0.6772274971008301),
 ('солнце', 0.6740930676460266),
 ('луны', 0.668319046497345),
 ('луна\xa0—', 0.6592304706573486),
 ('земля', 0.6425585150718689),
 ('венера', 0.6219674348831177),
 ('луной', 0.6106811761856079),
 ('марс\xa0—', 0.6072146892547607),
 ('солнца', 0.5985816717147827),
 ('юпитер\xa0—', 0.5922582745552063)]

##### Паттерн для парсинга новостей в вики-разметке

In [5]:
news_pattern = r'''(?x)\",\s\"URL[\w\W]+|[0-9]+\-[0-9]+\-[0-9]+
    |((?<=\s)\s)+|<[^>]*>|\[[^\]]*\]|[\\\"\'\n,:;\(\)]
    |[^а-яА-ЯёЁ%\"?!\n\.,\s\-\—:\(\)]+[,\.\n\\n\-\—:0-9\(\)\[\]]*''' #0-9

##### Блок для парсинга новостей из вики-разметки сайтов

In [6]:
def process(string, news_file):
    shorted_string = re.sub(r"\"type\"\s?:\s?\"news\".+", "", string)
    if (len(shorted_string) != len(string)):
        string = re.sub(news_pattern, "", string)
        string = re.sub(r"(\s(?=\s))*?", "", string)
        news_file.write(string.lower())
        news_file.write(" ")

raw_all_news_comments = ''
file_names = os.listdir(".")
print(file_names)
if not ("mined_texts" in file_names):
    os.mkdir("mined_texts")
if not ("results" in file_names):
    os.mkdir("results")
    
#TEST_FILE = open("TEST_FILE.txt", 'w', encoding = 'utf8')
for file_name in file_names:
    is_txt = re.search(r"\.post\.txt", file_name)
    if (not (is_txt == None)):
        if (not (file_name in ["facebook.post.txt", "vz.post.txt", "svpressa.post.txt", "echo.post.txt"])): 
        #на facebook и echo - только комментарии, на vz и svpressa - только блоги и комментарии
            news_file = open("mined_texts\\news_from_" + file_name, "w", encoding = "utf8")
            #TEST_FILE.write(file_name + '\n')
            raw_news_comments_file = open(file_name, "r", encoding = "utf8")
            chop = raw_news_comments_file.read(2*(10**8)) #читаем кусок в 200 мегабайт
            raw_line = ""
            index = 0
            while (chop != "") and (chop != None):
                index += 1
                if (len(raw_line) < 2*(10**6)):
                    raw_line += chop
                else:
                    raw_line = chop #too long
                splitted = re.split(r"\"plainText\"\s:\s\"|\"plainText\":\"", raw_line)
                if (len(splitted)!= 0):
                    for i in range(len(splitted) - 1):
                        process(splitted[i], news_file)
                    raw_line = splitted[len(splitted) - 1]
                chop = raw_news_comments_file.read(2*(10**6))
            print(file_name, index)
            news_file.close()
            raw_news_comments_file.close()            

['.ipynb_checkpoints', 'aif.post.txt', 'comments_task.ipynb', 'comments_task_large_files.ipynb', 'echo.post.txt', 'facebook.post.txt', 'gazeta.post.txt', 'inosmi.post.txt', 'Ivanov_Alexandr_1.ipynb', 'izvestia.post.txt', 'kommersant.post.txt', 'kp.post.txt', 'lenta.post.txt', 'LSA.ipynb', 'mined_texts', 'news_comments_mining-Copy1.ipynb', 'news_comments_mining.ipynb', 'ng.post.txt', 'PA_linreg_stochastic_grad_descent.ipynb', 'results', 'ria.post.txt', 'silverrain.post.txt', 'svpressa.post.txt', 'tales', 'test.txt', 'TEST_FILE.txt', 'Untitled.ipynb', 'Untitled1.ipynb', 'Untitled2.ipynb', 'vedomosti.post.txt', 'vz.post.txt']
aif.post.txt 1
gazeta.post.txt 1
inosmi.post.txt 1
izvestia.post.txt 1
kommersant.post.txt 1
kp.post.txt 1
lenta.post.txt 1
ng.post.txt 1
ria.post.txt 1
silverrain.post.txt 1
vedomosti.post.txt 1


###### Паттерн для парсинга комментариев в вики-разметке

In [45]:
comments_pattern = r'''(?x)\",\s\"URL[\w\W]+|\s(?=\s)
    |\\|\\n|&[^;]*;|<[^>]*>|\[[^\]]*\]|[0-9,]+(?=\s)
    |[^а-яА-ЯёЁ\%\"?!\n\.,\s\-\—:\(\)]+[,\.\s\n\\n\-\—:0-9\(\)]*'''

In [46]:
def space_after_punkt(matchobj):
    if matchobj.group(2) == ".": return (matchobj.group(1)+". ")
    if (matchobj.group(1) != "") and (matchobj.group(2) == "-"): return (matchobj.group(1)+"-")
    if (matchobj.group(1) == "") and (matchobj.group(2) == "-"): return " - "
    if (matchobj.group(1) != "") and (matchobj.group(2) == "—"): return (matchobj.group(1)+"—")
    if (matchobj.group(1) == "") and (matchobj.group(2) == "—"): return " — "
    if matchobj.group(2) == ",": return (matchobj.group(1)+", ")
    if matchobj.group(2) == ":": return (matchobj.group(1)+": ")
    else: return "";

###### Открываем файл lenta.post.txt, достаем из него комментарии, парсим их и записываем в файл.

In [47]:
lenta_raw_comments_file = open("lenta.post.txt", "r", encoding = "utf8")
lenta_raw_comments = lenta_raw_comments_file.read()

raw_comments_list = re.findall(r'''(?x)(?<=\"plainText\"\s:\s\").+?
    (?=\"type\"\s:\s\"comment\",\s\"status\"\s:\s\"regular\")''', lenta_raw_comments)
comments_file = open("mined_texts\\comments.txt", "w", encoding = "utf8")
for raw_comment in raw_comments_list:
    comment = re.sub(comments_pattern, "", raw_comment)
    #Здесь во всех случаях, когда - и — без пробелов разделяют слова, они выделяются пробелами(кроме случаев вроде "80-е").
    #это сделано потому, что комментаторы часто пропускают пробелы там, где они нужны. При этом мы также разделяем слова, которые
    #не должны быть разделены, но цена этого не так высока.
    comment = re.sub(r"([0-9]*)([\.,\—\-:](?=[^\s\.]))", space_after_punkt, comment)
    #Заменяем множественный символ конца предложения и смайлы на простую точку.
    comment = re.sub(r"[\?!\(\)]+|[:=;][\)\(\D\[\]]+", ". ", comment)
    comments_file.write(comment.lower())
    comments_file.write(" ")
comments_file.close()

###### Пишем токенайзер на основе word_tokenize из nltk. Если stem == True, выполняем стемминг токенов.(в тестовом режиме стемминг больше не производится. ВСЕ СЛОВА ПРИВОДЯТСЯ К НОРМАЛЬНОЙ ФОРМЕ.)

In [48]:
#from nltk.corpus import stopwords

def my_tokenizer(file_text, stem):
    file_text = re.sub(r"\.+", "$", file_text)
    tokens = nltk.word_tokenize(file_text.lower())
    #if stem == True:
    #    tokens = [stemmer.stem(i) for i in tokens]
    #stop_words = []
    #stop_words = stopwords.words('russian')
    #stop_words.extend(['что', 'это', 'так', 'вот', 'быть', 'как', 'в', 'к', 'на'])
    #tokens = [re.sub(r"|[0-9]+[,\.][0-9]+|[0-9]+|\.+", "",i) for i in tokens]
    #tokens = [i for i in tokens if ((i not in stop_words) and (re.search(r"[^а-яА-ЯёЁ$\-]", i) == None))]
    for i in range(len(tokens)):
        if re.search(r"[^а-яА-ЯёЁ$\-]", tokens[i]) != None:
            tokens[i] = "$"
        else:
            tokens[i] = morph.parse(tokens[i])[0].normal_form
    
    
    #print(tokens)
    #    if (len(token) == 1) and (token in ['а','и','в','у','','','']
    #tokens = [i.replace("«", "").replace("»", "") for i in tokens]
    return tokens
#i not in ["","…","`","«","»","“","„",',','``',"\"","'","''","-","–","—","?","...","!","%",":",";","(", ")",".."])\

In [49]:
print(my_tokenizer("Я люблю жопу... Мою жhпу h ^ хотят% телки.", False))

['я', 'любить', 'жопа', '$', 'мой', '$', '$', '$', 'хотеть', '$', 'телок', '$']


##### Токенизируем комментарии.

In [50]:
comments_file = open("mined_texts\\comments.txt", "r", encoding = "utf8")
untokenized_comments = comments_file.read()
comments_tokens = my_tokenizer(untokenized_comments, stem = False)

In [51]:
print(len(comments_tokens))

394487


In [52]:
class MySentences(object):
    def __init__(self, tokens):
        self.tokens = tokens
 
    def __iter__(self):
        i = 0
        while i < len(self.tokens):
            sentence = []
            while i < len(self.tokens) and self.tokens[i] =="$":
                i+=1
            while i < len(self.tokens) and self.tokens[i]!="$":
                sentence.append(self.tokens[i])
                i+=1
            yield(sentence)

In [36]:
print(comments_tokens[len(comments_tokens)-1])

$


In [53]:
for shit in MySentences(comments_tokens[3:100]):
    print(shit)

['русскоязычный', 'население']
['кишинёвнаш']
['ни', 'для', 'кто', 'не', 'секрет', 'что', 'у', 'власть', 'в', 'молдова', 'политика', 'купленый', 'москва', 'хватить', 'спамить']
['вот', 'когда', 'весь', 'увидеть', 'он']
['последствие']
['тогда', 'и', 'поговорить', 'и', 'ни', 'один', 'реиса', 'с', 'восток', 'кроме', 'константинополье', '-', 'так', 'откуда', 'терор', '-', 'или', 'вы', 'хотеть', 'сказать', 'что', 'из', 'параша', 'летать', 'гастербайтер', 'один']
['или', 'поставка', 'в', 'молдова', 'россииский', 'терорист']
['для', 'дестабилизация', 'ситуация', 'бессарабский', 'губерния', 'чё']
['ты', 'видимо', 'география', 'не', 'учить']
['у', 'молдаванин', 'нет', 'прямой', 'выход', 'к', 'море']
['идти', 'проспаться']
['от', 'сума', 'и', 'от', 'тюрьма', 'не', 'зарекаться']
['брейвика', 'в']


In [39]:
model = Word2Vec(sentences = MySentences(comments_tokens), min_count=0, window = 10, sg = 1, hs = 1, negative = 0)

In [54]:
model_stem = Word2Vec(sentences = MySentences(comments_tokens), min_count=0, window = 10, sg = 1, hs = 1, negative = 0)

In [74]:
model_norm = Word2Vec(sentences = MySentences(comments_tokens), min_count=7, window = 10, sg = 1, hs = 1, negative = 0)

In [55]:
model_stem.save('C:\\Users\\user\\Documents\\%HOME%\\Word2Vecs\\model1stemmed')

In [66]:
model.wv.most_similar("хуй")

KeyError: "word 'хуй' not in vocabulary"

In [65]:
model_stem.wv.most_similar(stemmer.stem("хуй"))

[('перегруз', 0.6806601285934448),
 ('мегаядерн', 0.6594175100326538),
 ('ракетонесущ', 0.6386466026306152),
 ('звиздун', 0.6290217638015747),
 ('ведьм', 0.6150438785552979),
 ('безобидн', 0.6143378615379333),
 ('стеснительн', 0.6141873598098755),
 ('заразител', 0.6136988997459412),
 ('хохслуслик', 0.611823320388794),
 ('кает', 0.6069585680961609)]

In [67]:
if "хуй" in comments_tokens:
    print("tru dat")

##### Токенизируем корпус новостей: блок для токенизации большого "чистого" корпуса.

In [32]:
news_tokens = []
file_names = os.listdir("mined_texts")
#Открываем файлы с новостями и комментариями, разбиваем текст на токены.
for file_name in file_names:
    #is_txt = re.search(r"news_from_[^\.]*\.post\.txt", file_name)
    #if not (is_txt == None) or (file_name == "idioma1-tokenized.txt"):
    if (file_name == "idioma1-tokenized.txt"):
        #print(file_name)
        news_file = open("mined_texts\\"+file_name, "r", encoding = "utf8")
        print("opened")
        #index = 0
        untokenized_news = news_file.read(2*(10**6)) #2*(10**6)
        while (untokenized_news != "") and (untokenized_news != None) and (len(news_tokens) < 10**7): #SHIT **7
            while(untokenized_news[len(untokenized_news)-1] not in [" ", "."]):
                letter = news_file.read(1)
                if (letter != ""):
                    #print("letter")
                    untokenized_news += letter
                else:
                    break #файл дочитан до конца
            #index += 1
            #print(index)
            news_tokens += my_tokenizer(untokenized_news, stem = False)
            #print("tok", len(news_tokens))
            untokenized_news = news_file.read(2*(10**4)) #2*(10**6)
            #print("*")
        print(file_name)

opened
idioma1-tokenized.txt


##### Блок для токенизации новостей, полученных из вики-разметки.

In [14]:
news_tokens = []
file_names = os.listdir("mined_texts")
#Открываем файлы с новостями и комментариями, разбиваем текст на токены.
for file_name in file_names:
    is_txt = re.search(r"news_from_[^\.]*\.post\.txt", file_name)
    if not ((is_txt == None) or (file_name == "idioma1-tokenized.txt")):
    #if (file_name == "idioma1-tokenized.txt"):
        print(file_name)
        news_file = open("mined_texts\\"+file_name, "r", encoding = "utf8")
        #print("opened")
        #index = 0
        untokenized_news = news_file.read(2*(10**6))
        while (untokenized_news != "") and (untokenized_news != None) and (len(news_tokens) < 10**7):
            while(untokenized_news[len(untokenized_news)-1] not in [" ", "."]):
                letter = news_file.read(1)
                if (letter != ""):
                    #print("letter")
                    untokenized_news += letter
                else:
                    break #файл дочитан до конца
            #index += 1
            #print(index)
            news_tokens += my_tokenizer(untokenized_news, stem = False)
            #print("tok", len(news_tokens))
            untokenized_news = news_file.read(2*(10**6))
            #print("*")
        #print(file_name)

news_from_aif.post.txt
news_from_gazeta.post.txt
news_from_inosmi.post.txt
news_from_izvestia.post.txt
news_from_kommersant.post.txt
news_from_kp.post.txt
news_from_lenta.post.txt
news_from_ng.post.txt
news_from_ria.post.txt
news_from_silverrain.post.txt
news_from_vedomosti.post.txt


In [9]:
test_str = ".Почему же критерий не работает. Он работает хуже, чем нужно. Почему же он не работает."
news_tokens = my_tokenizer(test_str, stem = False)
print(news_tokens)

['$', 'почему', 'же', 'критерий', 'не', 'работает', '$', 'он', 'работает', 'хуже', 'чем', 'нужно', '$', 'почему', 'же', 'он', 'не', 'работает', '$']


In [33]:
#Количество токенов из новостей
print(len(news_tokens))

10002618


In [32]:
print(news_tokens[4000:10000])

['не', 'только', 'любое', 'упоминание', 'наркотиков', 'вне', 'научного', 'просветительства', 'но', 'и', 'тема', 'наркомании', 'ассоциированная', 'с', 'известными', 'уважаемыми', 'людьми', 'прошлого', 'и', 'настоящего', 'какой-нибудь', 'фильм', 'типа', 'высоцкий', '$', 'спасибо', 'что', 'живой', 'или', 'просто', 'любыми', 'энергичными', 'добрыми', 'и/или', 'обаятельными', 'ребятами-киногероями', 'как', 'бы', 'ни', 'уверяли', 'нас', 'авторы', 'в', 'обратном', '$', 'дети', 'и', 'подростки', 'всегда', 'объект', 'влияния', 'и', 'воспитания', 'такова', 'природа', 'этого', 'возраста', '$', 'они', 'не', 'могут', 'без', 'авторитета', 'и', 'без', 'примера', 'для', 'подражания', '$', 'при', 'недостатке', 'или', 'отсутствии', 'внимания', 'со', 'стороны', 'родителей', 'других', 'родственников', 'педагогов', 'и', 'врачей', 'воспитателями', 'становятся', 'преступные', 'группы', 'и', 'наркопропагандисты', '$', 'голоса', 'за', 'легализацию', 'наркотиков', 'по', 'обыкновению', 'сопровождаются', 'упомина

##### Исследование, скольких сленговых слов нет в модели word2vec.

In [14]:
slang = 0
slang_not_in_w2v=0
watched = []
for word in comments_tokens:
    if stemmer.stem(word) not in watched:
        if stemmer.stem(word) not in news_tokens:
            slang += 1
            try:
                res = model[word]
            except KeyError:
                slang_not_in_w2v += 1
        watched.append(stemmer.stem(word))

In [15]:
print(slang)
print(slang_not_in_w2v)

6340
6081


##### Пишем генераторы n-грамм. Составляем словари, где ключи - n-граммы, значение - частота.

In [34]:
from collections import defaultdict
#ЗДЕСЬ ВМЕСТО ТОЧЕК СТОЯТ БАКСЫ. СЧИТАЕМ ИХ. УЧИТЫВАЕМ В МОДЕЛЯХ.
def get_unigrams(tokens_list):
    for i in range(len(tokens_list)):
        #if (tokens_list[i] != "."):
        yield tokens_list[i]

def get_bigrams(tokens_list):
    for i in range(len(tokens_list)-1):
        #if (tokens_list[i] != ".") and (tokens_list[i+1] != "."):
        yield tokens_list[i], tokens_list[i+1]
        
def get_trigrams(tokens_list):
    for i in range(len(tokens_list)-2):
        #if (tokens_list[i] != ".") and (tokens_list[i+1] != ".") and (tokens_list[i+2] != "."):
        yield tokens_list[i], tokens_list[i+1], tokens_list[i+2]
            
unigrams = {}
bigrams = {}
trigrams = {}

for a in get_unigrams(news_tokens):
    if (a) in unigrams:
        unigrams[a] += 1
    else:
        unigrams[a] = 1
        
for a, b in get_bigrams(news_tokens):
    #print(a,b)
    if (a, b) in bigrams:
        bigrams[a, b] += 1
    else:
        bigrams[a, b] = 1

for a, b, c in get_trigrams(news_tokens):
    if (a, b, c) in trigrams:
        trigrams[a, b, c] += 1
    else:
        trigrams[a, b, c] = 1

##### Обучаем модели би- и триграмма.

In [16]:
bigram_model = {}
trigram_model = {}
#Good-Turing discount: c* = (c+1)N_c+1/N_c, P* = c*/c (P = c/c)
for (t0, t1), freq in bigrams.items():
    if "$" not in (t0, t1):
        if (t0) in bigram_model:
            bigram_model[t0].update({t1 : freq/unigrams[t0]})
        else:
            bigram_model[t0] = {t1 : freq/unigrams[t0]}

for (t0, t1, t2), freq in trigrams.items():
    if "$" not in (t0, t1, t2):
        if (t0, t1) in trigram_model:
            trigram_model[t0, t1].update({t2 : freq/bigrams[t0, t1]})
        else:
            trigram_model[t0, t1] = {t2 : freq/bigrams[t0, t1]}

In [ ]:
def reverse(t0, t1):
    if (t1,t0) in bigrams:
        return bigrams[t1,t0]
    else:
        return 0

hi_bi_dict_smart = {}
N = len(news_tokens) - unigrams["$"]
bigrams_num = len(news_tokens) - 1
print("bigram num", bigrams_num)

for (t0, t1), freq in bigrams.items():
    if not "$" in (t0,t1):
        #print(t0,t1)
        if (unigrams[t0]>20) or (unigrams[t1]>20):
            t0_dot = 0
            t1_dot = 0
            if (t0, "$") in bigrams:
                t0_dot = bigrams[t0, "$"]
            if (t1, "$") in bigrams:
                t1_dot = bigrams[t1, "$"]
            O_11 = freq
            #print("O_11", O_11)

            O_12 = 2*unigrams[t0] - t0_dot - reverse(t0,"$") - bigrams[t0,t1] - reverse(t0,t1)
            #print("O_12", O_12)
            O_21 = 2*unigrams[t1] - t1_dot - reverse(t1,"$") - bigrams[t0,t1] - reverse(t0,t1)
            #print("O_21", O_21)
            O_22 = bigrams_num - O_12 - O_21 - O_11 - 2*unigrams["$"] + 2 #include/exclude formula LEN PIZDEC
            #print("O_22", O_22)
            if ((O_11+O_12)*(O_11+O_21)*(O_12+O_22)*(O_21+O_22)) == 0:
                hi_bi_dict_smart[t0, t1] = 0
            else:
                hi_bi_dict_smart[t0, t1] = (N*((O_11*O_22-O_21*O_12)**2))/((O_11+O_12)*(O_11+O_21)*(O_12+O_22)*(O_21+O_22))
        else:
            hi_bi_dict_smart[t0, t1] = -7

bigram num 10002617


In [ ]:
import operator
sorted_hi = sorted(hi_bi_dict_smart.items(), key=operator.itemgetter(1), reverse = True)
print(sorted_hi[:200])

In [22]:
print(unigrams['гаек'])

33


In [27]:
import operator
x = {1: 2, 3: 4, 4: 3, 2: 1, 0: 0}
sorted_x = sorted(x.items(), key=operator.itemgetter(1), reverse = True)
print(sorted_x[:2])

[(3, 4), (4, 3)]


In [13]:
print(sorted_hi[4000:4100])

[(('долговечную', 'эстетичную'), 2319095.4151661554), (('портсмуте', 'фор-марсель'), 2319095.4151661554), (('сшиванию', 'полотнищ'), 2319095.4151661554), (('лошадиным', 'дозам'), 2319095.4151661554), (('обескураживающую', 'непросвещенность'), 2319095.4151661554), (('кумачовый', 'пунцовый'), 2319095.4151661554), (('майдельшталь', 'манданштан'), 2319095.4151661554), (('кл', 'сукер'), 2319095.4151661554), (('карант', 'мангольство'), 2319095.4151661554), (('новоселки', 'лошницкой'), 2319095.4151661554), (('скрунда', 'кулдигского'), 2319095.4151661554), (('никраце', 'скрунде'), 2319095.4151661554), (('скрунде', 'кулдигский'), 2319095.4151661554), (('мастыни', 'салдусский'), 2319095.4151661554), (('нарративе', 'описывающим'), 2319095.4151661554), (('осам', 'муслимского'), 2319095.4151661554), (('совместеног', 'опарада'), 2319095.4151661554), (('анти-коминтерновскому', 'пакут'), 2319095.4151661554), (('таой', 'вопро'), 2319095.4151661554), (('звероватым', 'падишахом'), 2319095.4151661554), ((

##### Используя модель биграмма, ищем синонимы к второму слову из биграммы из комментариев, если второго слова нет в модели биграмма, а первое - есть.

In [23]:
def get_key(dictionary, value):
    answer = []
    for k, v in dictionary.items():
        if v == value:
            answer.append(k)
    return answer

#def unstem(dictionary, stemmed_word):
#    for k in dictionary.keys():
#        if stemmer.stem(k) == stemmed_word:
#            return stemmed_word
#    return None
#stemmed_unigrams = []

#for a in get_unigrams(comments_tokens):
#    stemmed_word = stemmer.stem(a)
#    if (stemmed_word) not in stemmed_unigrams:
#        stemmed_unigrams.append(stemmed_word)

stemmed_dict = {}

uni_from_bi_dict = {}
for a, b in get_bigrams(comments_tokens):
    if not (b in unigrams):
        if (a) in bigram_model:
            stemmed_word = stemmer.stem(b)
            if (stemmed_word) not in stemmed_dict.keys():
                stemmed_dict[stemmed_word] = b
                dict_input = b #dict_input - какой ключ мы положим в словарь 
            else:
                dict_input = stemmed_dict[stemmed_word]
            
            max_prob = max(bigram_model[a].values())
            likely_words = get_key(bigram_model[a], max_prob)
            if (dict_input) in uni_from_bi_dict:
                for word in likely_words:
                    if not (word in uni_from_bi_dict[dict_input]):
                        uni_from_bi_dict[dict_input].append(word)
            else:
                uni_from_bi_dict[dict_input] = likely_words

##### Используя модель триграмма, ищем синонимы к третьему слову из триграммы из комментариев, если третьего слова нет в модели, а первое и второе - есть.

In [24]:
uni_from_tri_dict = {}
for a, b, c in get_trigrams(comments_tokens):
    if not (c in unigrams):
        if (a, b) in trigram_model:
            stemmed_word = stemmer.stem(c)
            dict_input = stemmed_dict[stemmed_word] #мы уже точно один раз прошлись по всем комментариям и взяли stem.
            max_prob = max(trigram_model[a, b].values())
            likely_words = get_key(trigram_model[a, b], max_prob)
            if (dict_input) in uni_from_tri_dict:
                for word in likely_words:
                    if not (word in uni_from_tri_dict[dict_input]):
                        uni_from_tri_dict[dict_input].append(word)
            else:
                uni_from_tri_dict[dict_input] = likely_words

##### Распечатаем полученные словари в файлы:

In [26]:
def print_dict(dictionary, output_name):
    output = open("results\\"+output_name, "w", encoding = "utf8" )
    for key in sorted(dictionary):
        if (key != ""):
            output.write(key+' : ')
            for word in dictionary[key]:
                if (word != ""):
                    output.write(word+", ")
            output.write("\n")
    output.close()
print_dict(uni_from_bi_dict, "uni_from_bi.txt")
print_dict(uni_from_tri_dict, "uni_from_tri.txt")

##### Вопрос: как считать, что является сленгом, а что нет? Считаем, что тут поможет w2v на википедии, thus корпус новостей тоже содержит сленг + сложно токенизировать.

### Первый ансамбль:  проверяем, что сленг, а что нет, по w2v wiki. Далее, для сленга по w2v ищем синонимы, из них снова отбираем те, что есть в w2v wiki. Добавляем оттуда синонимы синонимов. Отдельно для сравнения методов ищем синонимы по N-граммам из корпуса новостей. 
Можно среди синонимов искать те, что вместе с предыдущим словом образуют биграмму. Можно считать также хи квадрат от них, и отдавать предпочтение тем, где он выше, если таковых несколько - ранжирование синонимов по качеству! Хотя как раз такое ранжирование и представляет выход w2v, если брать сверху вниз...
----  нельзя использовать wiki для выделения сленга по стемам, потому что она не стемовая. Качество - отстой. Можно брать для однокоренных слов синонимы и записывать их в одно поле словаря, внимание на повторяющиеся синонимы для однокоренных - сделано, теперь пайморфи. Идея: сленг встречается хотя бы несколько раз. Поэтому оставляем словарь юниграмм. Окей, теперь сленга на порядок меньше, он качественный, но синонимы - все равно говно. Что делать?

In [63]:
def get_unigrams(tokens_list):
    for i in range(len(tokens_list)):
        #if (tokens_list[i] != "."):
        yield tokens_list[i]
            
unigrams = {}

for a in get_unigrams(comments_tokens):
    if (a) in unigrams:
        unigrams[a] += 1
    else:
        unigrams[a] = 1
        


In [72]:
#watched = []
slang = []
for word in unigrams.keys():
    #print(word)
    if unigrams[word] >= 7:
        try:
            res = wiki[word]
        except KeyError:
            slang.append(word)
        watched.append(word)


In [75]:
#print(len(watched))
print(len(slang))

451


In [16]:
model_unstem = Word2Vec(sentences = MySentences(comments_tokens), min_count=0, window = 10, sg = 1, hs = 1, negative = 0)

In [76]:
vectors_norm = model_norm.wv
del model_norm

In [60]:
for word in vectors_norm.similar_by_word("лох"):
    print(word[0])

расселять
съесть
нескучно
обвести
угодный
зачмырил
дурак
названивать
козлы
ложа


In [77]:
w2v_dict = {}
for word in slang:
    if word == "$":
        continue
    syns = vectors_norm.similar_by_word(word)
    good_syns = []
    for syn in syns:
        try:
            res = wiki[syn[0]]
        except KeyError:
            continue
        good_syns.append(syn[0])
    w2v_dict[word] = good_syns

In [78]:
def print_dict(dictionary, output_name):
    output = open("results\\"+output_name, "w", encoding = "utf8" )
    for key in sorted(dictionary):
        if (key != ""):
            output.write(key+' : ')
            for word in dictionary[key]:
                if (word != ""):
                    output.write(word+", ")
            output.write("\n")
    output.close()
print_dict(w2v_dict, "w2v_dict_normed.txt")